# Introduction

Source: http://insideairbnb.com/get-the-data.html <br />
Data from: Rio de Janeiro, Rio de Janeiro, Brazil <br />

**Note:** <br />
* The listings dataset wasn't considered in this analysis as the filesize would exceed Github's max filesize limit <br />
* The reviews dataset had to be processed in order to fit the max filesize from Github
<br />

**Reviews_reduced creation script** 
<br />
<code>reviews.drop(['id', 'reviewer_name', 'reviewer_id'], axis = 1, inplace = True)
reviews.to_csv(data_filepath + 'reviews_reduced.csv')<code>

In [1]:
# System path setup
import os
import sys

print('Project root location in: \n' + os.path.abspath('')[:-9])

data_filepath = os.path.abspath('')[:-9] + '/data/'
sys.path.insert(0, os.path.abspath('')[:-9] + '/modules/')

Project root location in: 
/home/leonardo/Documents/udacity_projects/data_science_capstone_project/


In [2]:
def etl_needed(data_filepath = data_filepath):
    '''
    INPUTS:
    none
    OUTPUTS:
    etl_neecessity - (boolean) indicating if amenities_post_etl.csv and listings_post_etl.csv exist
    '''
    
    first_bool = os.path.isfile(data_filepath + 'amenities_post_etl.csv')
    second_bool = os.path.isfile(data_filepath + 'listings_post_etl.csv')
    
    if first_bool and second_bool:
        return False
    else:
        return True

In [3]:
# Importing data processing functions created on modules
from airbnb_data_processing import unpack_double_assessment
from airbnb_data_processing import neighbourhood_hdi_update_keys
from airbnb_data_processing import tokenize
from airbnb_data_processing import token_list_to_string
from airbnb_data_processing import literal_to_list_and_tokenize
from airbnb_data_processing import list_nan_columns
from airbnb_data_processing import compute_walking_dist_stations
from airbnb_data_processing import listings_assign_keys
from airbnb_data_processing import assign_station_score
from airbnb_data_processing import summarize_bathroom_string
from airbnb_data_processing import standardize_boolean_columns
from airbnb_data_processing import standardize_names
from airbnb_data_processing import compute_days_since_start

[nltk_data] Downloading package punkt to /home/leonardo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/leonardo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# Importing relevant liberaries for EDA
import pandas as pd
import numpy as np
import scipy

# Count Vectorizer for Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# Reading datasets
listings = pd.read_csv(data_filepath + 'listings.csv')

reviews = pd.read_csv(data_filepath + 'reviews_reduced.csv')

neighbourhood_hdi = pd.read_csv(data_filepath + 'neighbourhood_hdi.csv',
                                sep = ';',
                                decimal = ',',
                                encoding = 'utf-8')

/home/leonardo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Storing the geo-locations of metro statios in Rio de Janeiro
# Data obatined from Google Maps
# https://www.google.com/maps/search/estações+de+metro+rio+de+janeiros/@-22.8916528,-43.3403456,12z/data=!3m1!4b1
# Collected in 24/01/2022

metro_stations = {
    'central': (-22.9049171010074, -43.19083837044949),
    'gloria': (-22.920550514351582, -43.17648410807313),
    'estacio': (-22.913496579807873, -43.2064926626525),
    'botafogo': (-22.951082017707268, -43.18411168513209),
    'pavuna': (-22.806666225646197, -43.36493532847317),
    'cardeal_arcoverde' : (-22.96463512369287, -43.180656232771874),
    'iraja' : (-22.84796020759417, -43.32343657311362),
    'uruguaiana' : (-22.902940565469503, -43.18149436629887),
    'cinelandia': (-22.911063930197916, -43.175678735438794),
    'nova_america':(-22.879278506956446, -43.27187285300277),
    'coelho_neto': (-22.83150705338417, -43.34329456463904),
    'flamengo' : (-22.937231062820434, -43.17823192129637),
    'cidade_nova' : (-22.908770113484287, -43.20630803329488),
    'uruguai' : (-22.931835141573387, -43.23964451197971),
    'vicente_carvalho' : (-22.85399039798257, -43.31315765809618),
    'catete' : (-22.925787988052296, -43.17657983867064),
    'acesso_a_cardeal_arcoverde' : (-22.964675568740653, -43.18054869617871),
    'cantagalo' : (-22.976672769596224, -43.19385233439477),
    'afonso_pena' : (-22.91829820485412, -43.21782218194045),
    'general_osorio' : (-22.984766408179055, -43.19716911734364)
}

In [7]:
# Storing the geo-locations of metro statios in Rio de Janeiro
# Data obatined from Google Maps
# https://www.google.com/maps/search/estações+de+metro+rio+de+janeiros/@-22.8916528,-43.3403456,12z/data=!3m1!4b1
# Collected in 24/01/2022

metro_stations_google_score = {
    'central': 3.7,
    'gloria': 4.4,
    'estacio': 4.0,
    'botafogo': 4.0,
    'pavuna': 3.5,
    'cardeal_arcoverde' : 4.3,
    'iraja' : 3.8,
    'uruguaiana' : 4.0,
    'cinelandia': 4.4,
    'nova_america': 4.2,
    'coelho_neto': 3.7,
    'flamengo' : 4.3,
    'cidade_nova' : 4.2,
    'uruguai' : 4.4,
    'vicente_carvalho' : 4.1,
    'catete' : 4.3,
    'acesso_a_cardeal_arcoverde' : 4.3, # No score, but seems to be the same as cardeal_arcoverde
    'cantagalo' : 4.4,
    'afonso_pena' : 4.1,
    'general_osorio': 4.1 
}

# ETL - Extract, transform, load

## ETL: neighbourhood_hdi
**Download source:** https://www.data.rio/documents/58186e41a2ad410f9099af99e46366fd/about <br />
**Data source:** Dados básicos: IBGE-microdados dos Censos Demográficos 1991 e 2000.<br />
**Note:** <br/>
* To generate the csv file in the data folder, some formatting on Excel was performed translating the columns and removing columns not considered in this analysis.<br/>
* IBGE - Brazilian Institute of Geography and Statistics.<br/>
* Some neighbourhood names were adjusted in order to allow the retrieval of the HDI based on the neighbourhood names, some small research was done in order to increase the relieability of the data.

In [8]:
neighbourhood_hdi.head(n = 2)

,hdi_ranking,neighbourhood,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index
0,1,Gávea,2139.559275,0.924139,0.987167,1.0,0.970435
1,2,Leblon,2441.279121,0.907799,0.993403,1.0,0.967067


In [9]:
# In this IBGE study, some neighbourhoods were assessed together
# In such cases the different neighbourhoods are listed in the same row, separated by ","
print(neighbourhood_hdi.iloc[115])

hdi_ranking                                                   116
neighbourhood              Camorim, Vargem Pequena, Vargem Grande
per_capita_income_2000                                 279.093541
longevity_index                                          0.688297
education_index                                          0.836278
income_index                                             0.712834
human_development_index                                  0.745803
Name: 115, dtype: object


In [10]:
# Neighbourhood names unique values check
neighbourhood_unnique = neighbourhood_hdi['neighbourhood'].nunique() == neighbourhood_hdi['neighbourhood'].shape[0]
print('Neighbourhood column has only unique values: ' + str(neighbourhood_unnique))

Neighbourhood column has only unique values: True


In [11]:
# Dropping the HDI Ranking as it can be redundant given the fact that the DataFrame can be sorted when necessary
neighbourhood_hdi.drop('hdi_ranking', axis = 1, inplace = True)

# Unpacking double assessments 
neighbourhood_hdi = unpack_double_assessment(neighbourhood_hdi)

# Standardizing neighbourhood names
neighbourhood_hdi['neighbourhood'] = neighbourhood_hdi['neighbourhood'].apply(standardize_names)

# Updating neighbourhood keys
neighbourhood_hdi = neighbourhood_hdi_update_keys(neighbourhood_hdi)

/home/leonardo/.local/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [12]:
neighbourhood_hdi.head(n = 2)

,neighbourhood,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index,neighbourhood_key
0,gávea,2139.559275,0.924139,0.987167,1.0,0.970435,0
1,leblon,2441.279121,0.907799,0.993403,1.0,0.967067,1


## ETL: listings

In [13]:
listings.head(n = 2)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20211224070558,2021-12-25,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,...,4.9,4.76,4.66,NaN,f,1,1,0,0,1.92
1,24480,https://www.airbnb.com/rooms/24480,20211224070558,2021-12-25,"Nice and cozy near Ipanema Beach, w/ home office","My studio is located in the best of Ipanema, t...","The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,...,4.9,4.97,4.58,NaN,f,1,1,0,0,0.62


In [14]:
# Delisting columns according to project scope and column description
delisted_columns = [
                            'name',
                            'description',
                            'neighborhood_overview',
                            'listing_url',
                            'scrape_id',
                            'picture_url',
                            'host_picture_url',
                            'host_url',
                            'host_location',
                            'host_thumbnail_url',
                            'host_total_listings_count',
                            'host_name',
                            'host_about',
                            'host_neighbourhood',
                            'host_verifications',
                            'property_type', 
                            'availability_30',
                            'availability_60',
                            'availability_90',
                            'availability_365',
                            'instant_bookable',
                            'calculated_host_listings_count',
                            'calculated_host_listings_count_entire_homes',
                            'calculated_host_listings_count_private_rooms',
                            'calculated_host_listings_count_shared_rooms',
                            'minimum_nights',
                            'maximum_nights',
                            'minimum_minimum_nights',
                            'maximum_minimum_nights',
                            'minimum_maximum_nights',
                            'maximum_maximum_nights',
                            'has_availability',
                            'calendar_last_scraped',
                            'number_of_reviews_ltm',
                            'first_review',
                            'last_review',
                            'host_acceptance_rate',
                            'host_listings_count',
                            'number_of_reviews_l30d',
                            'number_of_reviews'
                            ]

In [15]:
# Hypothesis: listings.id is a column of primary key values
listings.id.nunique() == listings.shape[0]

True

In [16]:
# Evaluating possible redundant column
print(str((listings.host_listings_count == listings.host_total_listings_count).sum() == listings.shape[0]) + '\n')

diff_list = []
for control, test in zip(listings.host_listings_count, listings.host_total_listings_count):
    if control != test:
        diff_list.append((control, test))
        
print(diff_list)

# Conclusion: any of the 2 evaluated columnns can be dropped

False

[(nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan)]


In [17]:
# Preparing the neighbourhood name column to match the neighbourhoods from neighbourhood_hdi

# Standardizing neighbourhood names
listings['neighbourhood'] = listings.neighbourhood_cleansed.apply(standardize_names)

# Dropping the non standardized column
listings.drop(['neighbourhood_cleansed'], axis = 1, inplace = True)

# Assessing the neighbourhoods that could not be retrieved from the IBGE data
assess = []
for neighbourhood in listings.neighbourhood:
    if neighbourhood not in list(neighbourhood_hdi['neighbourhood']):
        assess.append(neighbourhood)

set(assess)

{'gericinó'}

In [18]:
# For 'gericinó'the missing data will be the average of the hdi table

# Creating a series with the neighbourhood data
gericino_case = pd.Series(data = 'gericinó', index = ['neighbourhood'])

# Calculating the mean for the numeric columns
gericino_series = neighbourhood_hdi.mean().append(gericino_case)

# Reindexing the series to append in neighbourhood_hdi
gericino_series.reindex(['neighbourhood',
                        'per_capita_income_2000',
                        'longevity_index', 
                        'education_index', 
                        'income_index', 
                        'human_development_index'])

# Appending new data 
neighbourhood_hdi = neighbourhood_hdi.append(gericino_series, ignore_index = True)

# Updating neighbourhood_hdi keys
neighbourhood_hdi = neighbourhood_hdi_update_keys(neighbourhood_hdi)

# Assigning keys to the neighbourhoods in listings.neighbourhoods
listings = listings_assign_keys(listings , neighbourhood_hdi)

/tmp/ipykernel_5465/1386082581.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  gericino_series = neighbourhood_hdi.mean().append(gericino_case)


In [19]:
if etl_needed(data_filepath) == True:
    '''
    LISTINGS ETL PIPELINE
    '''

    # Checking and dropping columns composed entirely of NaN values
    listings_columns_to_drop = list_nan_columns(listings)
    listings.drop(listings_columns_to_drop, axis = 1, inplace = True)

    # Dropping delisted columns, these columns were dropped as they were out of the scope of the analysis
    # given their definition
    listings.drop(delisted_columns, axis = 1, inplace = True)

    # Computing the days between when the host started and the last scraped date
    # This function can also drop (boolean argument) the given time columns
    listings = compute_days_since_start(listings, 'host_since', 'last_scraped', 'host_days_since_start', True)
    
    # Summarizing and replacing the bathrooms_text column
    bathroom_qty, shared_bathroom = summarize_bathroom_string(listings)
    listings['bathroom_qty'] = bathroom_qty
    listings['shared_bathroom'] = shared_bathroom
    listings.drop(['bathrooms_text'], axis = 1, inplace = True)

    # Standardizing boolean columns
    listings['host_has_profile_pic'] = listings['host_has_profile_pic'].apply(standardize_boolean_columns)
    listings['host_identity_verified'] = listings['host_identity_verified'].apply(standardize_boolean_columns)
    listings['host_is_superhost'] = listings['host_is_superhost'].apply(standardize_boolean_columns)

    # Converting literal lists to token strings
    listings['amenities'] = listings['amenities'].apply(literal_to_list_and_tokenize)

    # Computing the nearest stations
    walking_dist_stations, walking_dist_stations_scores = compute_walking_dist_stations(listings, 
                                                                                        metro_stations, 
                                                                                        metro_stations_google_score, 
                                                                                        walking_dist = 1)

    # Appending in the DataFrame
    listings['walking_dist_stations'] = walking_dist_stations
    listings['walking_dist_stations_scores'] = walking_dist_stations_scores

    # Dropping the latitude and longitude columns
    listings.drop(['latitude', 'longitude'], axis = 1, inplace = True)

    # Expanding the listings DataFrame
    listings_expanded = listings.join(neighbourhood_hdi, on = 'neighbourhood_key', how = 'left', lsuffix = '_hdi')

    # Expanding the listings DataFrame
    listings_expanded = listings.join(neighbourhood_hdi, on = 'neighbourhood_key', how = 'left', lsuffix = '_hdi')

    # Dropping the key columns and neighbourhood names
    listings_expanded.drop(['neighbourhood_key',
                            'neighbourhood_key_hdi',
                            'neighbourhood_hdi',
                            'neighbourhood'], axis = 1, inplace = True)

    '''
    LISTINGS AMENITIES COLUMN - BAG OF WORDS APPROACH
    '''

    vect = CountVectorizer(tokenizer=None)

    # Getting the counts for each token
    X = vect.fit_transform(listings_expanded['amenities'])

    # Converting to dense matrix to save
    X_dense = scipy.sparse.csr_matrix.todense(X)

    # Inverting the dictionary from {key : value} to {value : key}
    # Source: https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping
    matrix_vocabulary = {value: key for key, value in vect.vocabulary_.items()}

    # Retrieving the columns from the CountVectorizer matrix according to the generated vocabulary
    matrix_columns = []
    for i in range(0, len(matrix_vocabulary)):
        matrix_columns.append('amn_' + matrix_vocabulary[i])

    # Converting the dense matrix to DataFrame
    amenities_dataframe = pd.DataFrame(X_dense, columns = matrix_columns, index = listings_expanded.index)

    # Dropping amenities from listings_expanded
    listings_expanded.drop(['amenities'], axis = 1, inplace = True)

    # Saving the processed DataFrames as csv
    amenities_dataframe.to_csv(data_filepath + 'amenities_post_etl.csv')
    listings_expanded.to_csv(data_filepath + 'listings_post_etl.csv')

else:
    amenities_dataframe = pd.read_csv(data_filepath + 'amenities_post_etl.csv', index_col = 0)
    listings_expanded = pd.read_csv(data_filepath + 'listings_post_etl.csv', index_col = 0)

In [20]:
listings_expanded.head(n = 2)

,id,host_id,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,room_type,accommodates,bedrooms,...,host_days_since_start,bathroom_qty,shared_bathroom,walking_dist_stations,walking_dist_stations_scores,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index
0,17878,68997,within an hour,100%,0,1,1,Entire home/apt,5,2.0,...,4369.0,1.0,0,NONE,-1.0,1623.415521,0.879608,0.989888,1.0,0.956499
1,24480,99249,a few days or more,100%,0,1,1,Entire home/apt,2,1.0,...,4292.0,1.0,0,general_osorio,4.1,2465.445144,0.894594,0.991860,1.0,0.962151


In [21]:
amenities_dataframe.head(n = 2)

,amn_10,amn_10g,amn_10l,amn_10ml,amn_110v,amn_120l,amn_124,amn_14,amn_15,amn_17,...,amn_xampu,amn_xbox,amn_xiaomi,amn_xiomi,amn_xxx,amn_yamaha,amn_yamasterol,amn_year,amn_youtube,amn_yp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
